In [296]:
# Importing the libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as stats
import json
import copy

In [297]:
with open('../models/config.json', 'r') as f:
    config = json.load(f)

preprocess = config['preprocessing']

In [298]:
# Output Path
OUTPUT_PATH =preprocess['output_path']

# Input Path
INPUT_PATH = preprocess['input_path']

In [299]:
# Load the dataset
dataset = pd.read_csv(INPUT_PATH + '240624_validation_data.csv')


In [300]:
# Select the date for which you want to extract the data
date = '2023-06-26'

# Select the periode for which you want to extract the data
selected_period = 1 # 0 for day, 1 for week, 2 for month

# Number of scenarios to generate
num_scenarios = 1000
desired_num_scenarios = 10


In [301]:
# Extract the time, forecasted demand and actual demand
time = dataset['time']
time = pd.to_datetime(time, utc=True)
forecast_demand = dataset['predicted heat']
actual_demand = dataset['delivered heat']

# Create a dataframe with the necessary columns
df_data = pd.DataFrame({'time' : time, 'actual_demand': actual_demand, 'forecast_demand': forecast_demand})
df_data['hour']=time.dt.hour

# Calculate the error and absolute error
df_data['error']= df_data['actual_demand'] - df_data['forecast_demand']

# Calculate the mean of the error for whole dataset
mu_all = df_data['error'].mean()
sigma_all = df_data['error'].std()
print('Mean of of the error of each hour:', mu_all)
print('Standard deviation of the mean of the error of each hour:', sigma_all)

df_data


Mean of of the error of each hour: -8.39020298257356
Standard deviation of the mean of the error of each hour: 31.42172005784749


,time,actual_demand,forecast_demand,hour,error
0,2023-03-02 16:00:00+00:00,254.990005,255.590651,16,-0.600646
1,2023-03-02 17:00:00+00:00,187.787503,237.501176,17,-49.713673
2,2023-03-02 18:00:00+00:00,230.629171,264.220971,18,-33.591800
3,2023-03-02 19:00:00+00:00,233.329169,236.491860,19,-3.162691
4,2023-03-02 20:00:00+00:00,222.775003,216.179837,20,6.595167
...,...,...,...,...,...
2996,2023-07-12 19:00:00+00:00,55.804167,51.912949,19,3.891219
2997,2023-07-12 20:00:00+00:00,14.191667,37.334409,20,-23.142742
2998,2023-07-12 21:00:00+00:00,22.533334,37.278034,21,-14.744700
2999,2023-07-12 22:00:00+00:00,15.287500,37.703201,22,-22.415700


In [302]:
# Calculate the mean and standard deviation of the error of each hour of a day. 
error = df_data.groupby('hour')['error'].agg(['mean', 'std']).reset_index()
error  = error.rename(columns={'mean': 'mu', 'std': 'sigma'})

# Save the error data to a dictionary
error_dict = error.set_index('hour').to_dict(orient='index')

# Print the error dictionary
error_dict



{0: {'mu': -12.731658021190215, 'sigma': 25.018753157221795},
 1: {'mu': -8.717701939869812, 'sigma': 25.787822594592562},
 2: {'mu': 38.62809145641247, 'sigma': 40.06221098717317},
 3: {'mu': -11.219355528775246, 'sigma': 35.89610812413864},
 4: {'mu': 4.6619634940865975, 'sigma': 29.89150310430875},
 5: {'mu': -3.1503448513536596, 'sigma': 30.62503489088334},
 6: {'mu': -11.665524616604312, 'sigma': 28.687483429676817},
 7: {'mu': -8.965185204385374, 'sigma': 30.89163216771525},
 8: {'mu': -4.254730131452734, 'sigma': 29.0944311173442},
 9: {'mu': -9.849400683302798, 'sigma': 33.982800569041345},
 10: {'mu': -16.93716965553407, 'sigma': 31.972997351520558},
 11: {'mu': -11.808679656691835, 'sigma': 27.301664271959485},
 12: {'mu': -14.417149987864374, 'sigma': 29.132160670687032},
 13: {'mu': -11.036739802937781, 'sigma': 27.055038468354763},
 14: {'mu': -14.235238982654698, 'sigma': 29.627671262886945},
 15: {'mu': -12.303575238253774, 'sigma': 33.18363957415679},
 16: {'mu': -7.271

In [303]:
# Convert selected_date to datetime with UTC timezone
selected_date = date
selected_date = pd.to_datetime(selected_date, utc=True)

# Determine the start and end dates based on the selected period
if selected_period == 0:  # Day
    start_date = selected_date.normalize()
    end_date = start_date + pd.Timedelta(hours=23)
elif selected_period == 1:  # Week
    start_date = selected_date.normalize()
    end_date = start_date + pd.Timedelta(days=6, hours=23)
elif selected_period == 2:  # Month
    start_date = selected_date.normalize()
    end_date = (start_date + pd.DateOffset(months=1)) - pd.Timedelta(hours=1)
else:
    raise ValueError("selected_period must be 0 (day), 1 (week), or 2 (month)")

# Filter data for the selected period
period_data = df_data[(df_data['time'] >= start_date) & (df_data['time'] <= end_date)].reset_index(drop=True)
period_data = period_data[['time', 'hour', 'actual_demand', 'forecast_demand', 'error']]

# Generate expected timestamps
expected_time = pd.date_range(
    start=start_date,
    end=end_date,
    freq='H',
    tz='UTC'
)

# Check for missing timestamps
missing_time = expected_time.difference(period_data['time'])

print('Expected number of data points:', len(expected_time))
print('Actual number of data points:', len(period_data))

if missing_time.empty:
    print('The selected period has all the expected data.')
else:
    print(f'The selected period is missing {len(missing_time)} data points.')
    print('Missing timestamps:')
    print(missing_time)

# Display the filtered data
period_data

Expected number of data points: 168
Actual number of data points: 168
The selected period has all the expected data.


C:\Users\Paul Wilhelm\AppData\Local\Temp\ipykernel_36572\1721325174.py:23: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_time = pd.date_range(


,time,hour,actual_demand,forecast_demand,error
0,2023-06-26 00:00:00+00:00,0,4.770833,38.603098,-33.832264
1,2023-06-26 01:00:00+00:00,1,42.383334,46.163916,-3.780582
2,2023-06-26 02:00:00+00:00,2,178.750003,90.275907,88.474096
3,2023-06-26 03:00:00+00:00,3,36.850000,61.863150,-25.013150
4,2023-06-26 04:00:00+00:00,4,68.887502,66.026400,2.861101
...,...,...,...,...,...
163,2023-07-02 19:00:00+00:00,19,46.175001,53.784231,-7.609229
164,2023-07-02 20:00:00+00:00,20,7.100000,38.931915,-31.831915
165,2023-07-02 21:00:00+00:00,21,13.379167,40.324308,-26.945141
166,2023-07-02 22:00:00+00:00,22,11.204167,39.418925,-28.214758


---

In [304]:
# Extract the data for the selected date
daily_date = df_data[df_data['time'].dt.date == pd.to_datetime(selected_date).date()].reset_index(drop=True)
daily_date = daily_date[['time', 'hour', 'actual_demand', 'forecast_demand', 'error']]
if(len(daily_date)==24):
    print('The selected date has 24 hours')
else:
    print('The selected date does not have 24 hours')
daily_date


The selected date has 24 hours


,time,hour,actual_demand,forecast_demand,error
0,2023-06-26 00:00:00+00:00,0,4.770833,38.603098,-33.832264
1,2023-06-26 01:00:00+00:00,1,42.383334,46.163916,-3.780582
2,2023-06-26 02:00:00+00:00,2,178.750003,90.275907,88.474096
3,2023-06-26 03:00:00+00:00,3,36.850000,61.863150,-25.013150
4,2023-06-26 04:00:00+00:00,4,68.887502,66.026400,2.861101
5,2023-06-26 05:00:00+00:00,5,52.920834,64.045179,-11.124345
6,2023-06-26 06:00:00+00:00,6,29.887500,62.188697,-32.301197
7,2023-06-26 07:00:00+00:00,7,24.229167,57.766094,-33.536926
8,2023-06-26 08:00:00+00:00,8,48.975001,55.991011,-7.016010
9,2023-06-26 09:00:00+00:00,9,62.204168,50.229955,11.974214


---

## Methode: Monte Carlo Sampling and Roulette Wheel Mechanism


In [305]:
import numpy as np
import random
from scipy import stats

# Number of intervals
num_intervals = 7

# Function to randomly select an interval based on the CDF
def select_interval(cumulative_probabilities):
    rnd = random.random()
    for i, cp in enumerate(cumulative_probabilities):
        if rnd <= cp:
            return i
    return len(cumulative_probabilities) - 1

# Get the list of dates in the selected period
date_range = pd.date_range(start=start_date.normalize(), end=end_date.normalize(), freq='D', tz='UTC')

# Initialize dictionaries to store scenario data
scenario_values = {f"Scenario{idx+1}": [] for idx in range(num_scenarios)}
scenario_probs = {f"Scenario{idx+1}": 1 for idx in range(num_scenarios)}

# Generate scenarios for each day
for single_date in date_range:
    # Extract data for the current day
    day_data = period_data[period_data['time'].dt.normalize() == single_date].reset_index(drop=True)
    
    if len(day_data) != 24:
        print(f"Warning: {single_date.date()} does not have 24 hours of data. Skipping this day.")
        continue  # Skip this day or handle as needed
    
    # Generate scenarios for this day
    daily_scenarios = []
    daily_probabilities = []
    
    for scenario_index in range(num_scenarios):
        scenario_name = f"Scenario{scenario_index + 1}"
        scenario = []
        scenario_probability = 1.0  # Initialize the scenario probability
        
        for t in range(24):  # 24 hours in a day
            # Get the hour for the current time step
            hour = day_data['hour'].iloc[t]
            
            # Fetch mu and sigma for the current hour
            mu = error_dict[hour]['mu']
            sigma = error_dict[hour]['sigma']
            
            # Define the intervals
            intervals = np.linspace(mu - 3 * sigma, mu + 3 * sigma, num_intervals + 1)
            interval_centers = (intervals[:-1] + intervals[1:]) / 2
            
            # Calculate probabilities for each interval
            probabilities = [
                stats.norm.cdf(intervals[i + 1], mu, sigma) - stats.norm.cdf(intervals[i], mu, sigma)
                for i in range(len(intervals) - 1)
            ]
            cum_probabilities = np.cumsum(probabilities)
            
            # Select interval index based on cumulative probabilities
            interval_index = select_interval(cum_probabilities)
            
            # Calculate forecast error and add to forecast demand
            forecast_error = interval_centers[interval_index]
            actual_demand = day_data['forecast_demand'].iloc[t] + forecast_error
            scenario.append(actual_demand)
            
            # Multiply the probability of this interval to the scenario probability
            scenario_probability *= probabilities[interval_index]
        
        # Append the scenario values and probabilities for this day
        daily_scenarios.append(scenario)
        daily_probabilities.append(scenario_probability)
    
    # Normalize the daily probabilities
    total_daily_probability = sum(daily_probabilities)
    daily_probabilities = [p / total_daily_probability for p in daily_probabilities]
    
    # Combine daily scenarios into full-period scenarios
    for scenario_index in range(num_scenarios):
        scenario_name = f"Scenario{scenario_index + 1}"
        scenario_values[scenario_name].extend(daily_scenarios[scenario_index])
        scenario_probs[scenario_name] *= daily_probabilities[scenario_index]

# Normalize the scenario probabilities over the entire period
total_period_probability = sum(scenario_probs.values())
for scenario_name in scenario_probs:
    scenario_probs[scenario_name] /= total_period_probability

# Prepare the final data
heat_demand_data = {}
for scenario_name in scenario_values:
    heat_demand_data[scenario_name] = {
        "Probability": scenario_probs[scenario_name],
        "Values": scenario_values[scenario_name]
    }

# Output the scenarios and their normalized probabilities
for scenario_name, data in heat_demand_data.items():
    print(f"{scenario_name}:")
    print(f"Normalized Probability: {data['Probability']:.6f}")
    print(f"Values: {data['Values']}\n")


Scenario1:
Normalized Probability: 0.000000
Values: [68.76073072186162, 81.65390993375527, 163.24303636076635, 19.875702035536335, 70.68836375127425, 8.394774129263183, 25.93390124778211, 22.322366635072974, 51.73628070822568, 40.380553877503864, 57.41855772344896, 54.33958361798484, 55.705926557574536, 16.16416984813666, 64.1956343821465, 71.9483150315671, 79.03933793590109, 86.77317765146898, 14.60731972719801, 66.25964818547268, 15.317672731983397, 1.9961596844406841, 16.461206263821154, 40.08809430511352, 50.06284060211339, 84.35490954734769, 165.46918523961995, 25.10031629870116, 103.20738209579243, 64.41492323493648, 51.136584782270894, -4.0132892947776355, 98.51817241620566, 68.37553326349277, 31.16368271837608, 31.046921714256854, 57.71651815297655, 39.63694096167053, -32.68542714688904, 51.446922728317844, 26.59362945774683, 7.945149284646078, 63.472266756705345, 68.8345603875247, 20.575661994122886, 3.8704016516798916, 0.18680264996309148, 3.04047860150758, 10.83656340294538,

In [306]:
# Periodenbezeichnung definieren
period_str = {0: 'day', 1: 'week', 2: 'month'}

# Datumsstrings für den Dateinamen vorbereiten
start_date_str = start_date.strftime('%Y%m%d')
end_date_str = end_date.strftime('%Y%m%d')
filename_date = f"{start_date_str}_to_{end_date_str}_{period_str[selected_period]}"

# Erstellen des forecast_demand_dict mit der gewünschten Struktur
forecast_demand_dict = {
    "heat_demand": {
        str(t + 1): float(demand)
        for t, demand in enumerate(period_data['forecast_demand'])
    }
}

# Speichern des Forecast Demand in einer JSON-Datei
with open(f'{OUTPUT_PATH}heat_demand_{filename_date}.json', 'w') as json_file:
    json.dump(forecast_demand_dict, json_file, indent=4)

# Anpassen von heat_demand_data für die gewünschte Struktur
# heat_demand_scenarios = {}
# for scenario_name, data in heat_demand_data.items():
#     scenario_dict = {"Probability": data["Probability"]}
#     scenario_dict.update({
#         str(t + 1): float(value)
#         for t, value in enumerate(data["Values"])
#     })
#     heat_demand_scenarios[scenario_name] = scenario_dict

# Speichern der Szenarien in einer JSON-Datei
# with open(f'{OUTPUT_PATH}heat_demand_scenarios_{filename_date}.json', 'w') as json_file:
#     json.dump(heat_demand_scenarios, json_file, indent=4)

# # Laden der Daten bei Bedarf
# with open(f'{OUTPUT_PATH}heat_demand_{filename_date}.json', 'r') as json_file:
#     loaded_forecast_demand = json.load(json_file)

# with open(f'{OUTPUT_PATH}heat_demand_scenarios_{filename_date}.json', 'r') as json_file:
#     loaded_scenarios = json.load(json_file)


In [307]:
print(start_date)
print(end_date)

2023-06-26 00:00:00+00:00
2023-07-02 23:00:00+00:00


## Szenarien Reduktion

### Backward Scenario Reduction Methode

In [308]:
import numpy as np
import copy

# Copy the scenario data
S = copy.deepcopy(heat_demand_data)  # Current scenarios
DS = {}  # Deleted scenarios

# Extract the scenario names
scenario_names = list(S.keys())
num_scenarios = len(scenario_names)

# Extract the number of time periods from one of the scenarios
time_periods = range(len(S[scenario_names[0]]["Values"]))

# Precompute the distances between all pairs of scenarios
distance_matrix = np.zeros((num_scenarios, num_scenarios))

for i in range(num_scenarios):
    for j in range(i + 1, num_scenarios):
        s1 = S[scenario_names[i]]["Values"]
        s2 = S[scenario_names[j]]["Values"]
        # Compute the Euclidean distance
        distance = np.sqrt(sum((v1 - v2) ** 2 for v1, v2 in zip(s1, s2)))
        distance_matrix[i, j] = distance
        distance_matrix[j, i] = distance  # Symmetric matrix

# Perform the scenario reduction
while len(S) > desired_num_scenarios:
    # Step 2: For each scenario nk, determine nr with minimum distance DTk,r
    nearest_neighbor = {}
    for k_idx in range(num_scenarios):
        k_name = scenario_names[k_idx]
        if k_name not in S:
            continue
        distances = [
            (distance_matrix[k_idx, s_idx], scenario_names[s_idx])
            for s_idx in range(num_scenarios)
            if scenario_names[s_idx] in S and scenario_names[s_idx] != k_name
        ]
        if distances:
            min_distance, r_name = min(distances, key=lambda x: x[0])
            nearest_neighbor[k_name] = (min_distance, r_name)
    
    # Step 3: Calculate PDk,r = Pr(k) * DTk,r
    PD = {}
    for k_name in nearest_neighbor:
        Pr_k = S[k_name]["Probability"]
        DTk_r, r_name = nearest_neighbor[k_name]
        PD_k_r = Pr_k * DTk_r
        PD[k_name] = (PD_k_r, r_name)
    
    # Select d for which PDd = min PDk
    d_name, (min_PDd, r_name) = min(PD.items(), key=lambda x: x[1][0])
    
    # Step 4: Remove nd from S, add nd to DS, adjust probabilities
    Pr_d = S[d_name]["Probability"]
    Pr_r = S[r_name]["Probability"]
    S[r_name]["Probability"] = Pr_r + Pr_d
    DS[d_name] = S.pop(d_name)
    
    # Update the scenario names and number of scenarios
    scenario_names = list(S.keys())
    num_scenarios = len(scenario_names)

# Normalize the probabilities again to ensure they sum to 1
total_probability = sum(scenario["Probability"] for scenario in S.values())
for scenario in S.values():
    scenario["Probability"] /= total_probability

# Output the reduced scenarios and their probabilities
print("\nReduced Scenarios:")
for scenario_name, data in S.items():
    print(f"{scenario_name}:")
    print(f"  Normalized Probability: {data['Probability']:.4f}")
    print(f"  Values: {data['Values']}")



Reduced Scenarios:
Scenario74:
  Normalized Probability: 0.0013
  Values: [47.316085158528665, 37.44621405731088, 25.886884404744052, 112.17998006903571, 96.30965212639605, 60.89483394220605, 75.11244427008523, -4.15617522296867, 101.6124483379586, 11.252439104039858, 30.013131422145626, 7.536730580340013, 105.64677342160945, 62.54423579388769, 64.1956343821465, 43.50519539657556, 79.03933793590109, 114.60335870872564, 85.17520288012449, 45.78120903668678, -2.9834486280062436, 36.095362993612085, 36.631603239444345, 40.08809430511352, 28.61819503878042, 62.25106160912551, 62.45207127260321, 55.86840897653428, 77.58609372067063, 143.1650129543508, 51.136584782270894, 48.943794421305654, 48.64200478647274, 68.37553326349277, 3.758256417072751, 7.645495195434442, 32.74609472095909, 39.63694096167053, 18.10486644663144, 108.33316199830092, 82.76845908589362, 35.77533034190273, 16.427011321421027, 48.35612123873879, 2.2745406341332455, -13.179200002905823, 0.18680264996309148, 83.895577597

In [309]:
# Create a dictionary to store the reduced scenarios
reduced_heat_demand_scenarios = {}
for scenario_name, data in S.items():
    scenario_dict = {"Probability": data["Probability"]}
    scenario_dict.update({
        str(t + 1): float(value)
        for t, value in enumerate(data["Values"])
    })
    reduced_heat_demand_scenarios[scenario_name] = scenario_dict

# Save the reduced scenarios to a JSON file
with open(f'{OUTPUT_PATH}reduced_heat_demand_scenarios_{filename_date}.json', 'w') as json_file:
    json.dump(reduced_heat_demand_scenarios, json_file, indent=4)
